# Laboratorio 09

Integrantes:
- Ricardo Méndez 21289
- Sara Echeverría 21371
- Francisco Castillo 21562
- Melissa Pérez 21385

Repositorio: https://github.com/MelissaPerez09/Laboratorio09-CC3045

## Task 01 - Teoría

### 1. Diga cual es la diferencia entre Modelos de Markov y Hidden Markov Models

### 2. Investigue qué son los factorial HMM (Hidden Markov Models)

### 3. Especifique en sus propias palabras el algoritmo Forward Backward para HMM

### 4. En el algoritmo de Forward Backward, por qué es necesario el paso de Backward (puede escribir ejemplos o casos para responder esta pregunta)

## Task 02 - Algoritmo Forward Backward en HMM

En este ejercicio estamos ante un modelo meteorológico representado por un Modelo Oculto de Markov (HMM) con dos estados: "Soleado" y "Lluvioso". Queremos predecir el tiempo en un día determinado basándonos en las observaciones de si el día anterior estuvo soleado o lluvioso.

In [ ]:
class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        # Inicializar parámetros del HMM

    def generate_sequence(self, length):
        # Generar una secuencia de observaciones basada en el HMM.

    def forward(self, observations):
        # Implementar el paso adelante del algoritmo hacia atrás-adelante

    def backward(self, observations):
        # Implementar el paso hacia atrás del algoritmo hacia atrás-adelante

    def compute_state_probabilities(self, observations):
        # Combine probabilidades hacia adelante y hacia atrás para calcular probabilidades de estado

# Uso y datos
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Sunny', 'Rainy']
initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}
transition_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}}
emission_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}}

hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

# Generar una secuencia de observaciones.
obs_sequence = hmm.generate_sequence(5)
print("Secuencia Generada:", obs_sequence)

# Cálculo de probabilidades forward
forward_probs = hmm.forward(observations)
print("\nProbabilidades Forward:")
print(forward_probs)

# Cálculo de probabilidades backward
backward_probs = hmm.backward(observations)
print("\nProbabilidades Backward:")
print(backward_probs)

# Calcular probabilidades de estado
state_probs = hmm.compute_state_probabilities(observations)
print("\nProbabilidades de Estados:")
print(state_probs)